In [55]:
# l=[pd.NA for i in range(10)]
# import numpy as np
# s=np.random.randint(1,5,(5,)).tolist()
# x=np.random.randint(1,5,(5,)).tolist()
# l=s+l+x+l
# l=pd.Series(l)
# l=pd.to_numeric(l)

# l=l.interpolate(method='linear', limit=3, limit_direction='both', limit_area='inside')
# l

In [56]:
import pandas as pd

def clean(df, limit_days, feature_threshold):
    df['Timestamp']=pd.to_datetime(df['Timestamp'])
    df=df.set_index('Timestamp')
    start_date=df.index[0]
    end_date=df.index[-1]
    full_index=pd.date_range(start_date,end_date, freq='1h')
    df=df.reindex(full_index)
    limit_hours=24*limit_days
    df=df.interpolate(method='linear', limit=limit_hours, limit_direction='both', limit_area='inside', axis=0)
    df=df.dropna(axis=0, how='all')
    num_rows=df.shape[0]
    feature_present_fraction=df.notnull().sum(axis=0)/num_rows
    df = df.drop(columns=feature_present_fraction[feature_present_fraction < feature_threshold].index)
    df= df.dropna(axis=0, how='any')
    return df

In [102]:
df=pd.read_csv(r"/home/rishi/ML Projects/Air Pollution/sites_comb/site_113_Shadipur_Delhi_CPCB_15Min.csv")
df.shape

(140256, 25)

In [103]:
x=clean(df,limit_days=7, feature_threshold=0.7)

In [104]:
x.shape
x.to_csv("x_v1.csv")

In [105]:
x.shape

(80650, 15)

In [112]:
def num_gaps(df):
    time_diffs = df.index.to_series().diff()
    expected_freq = pd.Timedelta(hours=1)
    is_gap = time_diffs > expected_freq
    group_ids = is_gap.cumsum()

    # Calculate duration for each group using agg
    group_durations = df.groupby(group_ids).apply(
        lambda g: (g.index.max() - g.index.min()).total_seconds() / (24 * 3600),
    )
    
    value_counts = group_durations.value_counts().sort_index()
    
    return value_counts, sum(value_counts)

In [113]:
g=num_gaps(x)
g

(4.500000       1
 8.791667       1
 14.000000      5
 14.041667      2
 14.083333      1
 14.125000      1
 14.166667      1
 14.291667      1
 15.458333      1
 22.083333      1
 22.625000      1
 23.791667      1
 26.125000      1
 34.291667      1
 36.250000      1
 41.083333      1
 42.166667      1
 50.625000      1
 56.416667      1
 65.750000      1
 68.250000      1
 70.916667      1
 72.625000      1
 80.000000      1
 82.541667      1
 84.000000      1
 100.291667     1
 103.083333     1
 103.958333     1
 116.875000     1
 125.541667     1
 182.750000     1
 312.833333     1
 1250.416667    1
 Name: count, dtype: int64,
 39)

In [ ]:
def num_gaps(df):
    time_diffs = df.index.to_series().diff()
    expected_freq = pd.Timedelta(hours=1)
    is_gap = time_diffs > expected_freq
    group_ids = is_gap.cumsum()

    # Calculate duration for each group using agg
    group_durations = df.groupby(group_ids).apply(
        lambda g: (g.index.max() - g.index.min()).total_seconds() / (24 * 3600),
    )
    
    value_counts = group_durations.value_counts().sort_index()
    
    return value_counts, sum(value_counts)

In [116]:
import os

def save_contiguous_series(df, file_name):
    # Create folder with the file_name
    os.makedirs(file_name, exist_ok=True)
    
    # Identify gaps and create group IDs
    time_diffs = df.index.to_series().diff()
    expected_freq = pd.Timedelta(hours=1)
    is_gap = time_diffs > expected_freq
    group_ids = is_gap.cumsum()
    
    # Save each contiguous group
    for group_id, group_df in df.groupby(group_ids):
        start_time = group_df.index.min().strftime('%Y%m%d_%H%M%S')
        end_time = group_df.index.max().strftime('%Y%m%d_%H%M%S')
        
        # Reset index and put timestamp in 'Timestamp' column
        group_df_reset = group_df.reset_index()
        group_df_reset.rename(columns={'index': 'Timestamp'}, inplace=True)
        
        output_filename = f"{file_name}/{file_name}_{start_time}_to_{end_time}.csv"
        group_df_reset.to_csv(output_filename, index=False)
        
    print(f"Saved {len(df.groupby(group_ids))} contiguous series to folder '{file_name}/'")
    
    return len(df.groupby(group_ids))

In [117]:
save_contiguous_series(x, "temp_split")

Saved 39 contiguous series to folder 'temp_split/'


39